<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Keras_Custom_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf 
from tensorflow import keras

# Custom Layer with Subclassing API

In [2]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()

    w_init= tf.random_normal_initializer()
    self.w=tf.Variable(
        initial_value=w_init(shape=(input_dim,units), dtype="float32"),
        trainable=True 
    )
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(units,), dtype="float32"),
        trainable=True
    )
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [3]:
x = tf.ones((2,2))
x 

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
linear_layer = Linear(4,2)
y = linear_layer(x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.06356013, -0.02136349,  0.0958024 ,  0.04870867],
       [ 0.06356013, -0.02136349,  0.0958024 ,  0.04870867]],
      dtype=float32)>

In [5]:
linear_layer.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.08850016, -0.0294551 ,  0.04489021,  0.01145601],
        [-0.02494004,  0.0080916 ,  0.0509122 ,  0.03725266]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

# Custom Layer using add_weight

In [6]:
from tensorflow.python.ops.init_ops_v2 import Initializer
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()
    self.w = self.add_weight(shape=(input_dim, units),
                             initializer="random_normal",
                             trainable = True)
    self.b = self.add_weight(shape=(units),
                             initializer = "zeros",
                             trainable = True)
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b  

In [7]:
x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)

In [8]:
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.07724835, 0.0008499 , 0.106326  , 0.02716313],
       [0.07724835, 0.0008499 , 0.106326  , 0.02716313]], dtype=float32)>

# Custom Layer without the shape of the inputs

In [13]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32):
    super(Linear,self).__init__()
    self.units = units
  def build(self,input_shape):
    self.w = self.add_weight(shape=(input_shape[-1],self.units),
                             initializer = "random_normal",
                             trainable = True)
    self.b = self.add_weight(shape = (self.units,),
                             initializer = "random_normal",
                             trainable = True)
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b               

In [14]:
linear_layer = Linear(32)
y = linear_layer(x)
y

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[ 0.12529299, -0.0561902 ,  0.03834991, -0.07893594,  0.00859061,
         0.08317355, -0.05678261,  0.0037364 , -0.04070574, -0.12113686,
         0.03729274,  0.01368941, -0.05131226,  0.03087646, -0.02225749,
         0.04070945,  0.01450459,  0.08193067,  0.0622445 ,  0.1920329 ,
        -0.0106803 , -0.07432184,  0.003447  ,  0.11814891, -0.07613041,
         0.01972482, -0.08320747,  0.13414364,  0.00047702, -0.15275721,
         0.09923995, -0.06166402],
       [ 0.12529299, -0.0561902 ,  0.03834991, -0.07893594,  0.00859061,
         0.08317355, -0.05678261,  0.0037364 , -0.04070574, -0.12113686,
         0.03729274,  0.01368941, -0.05131226,  0.03087646, -0.02225749,
         0.04070945,  0.01450459,  0.08193067,  0.0622445 ,  0.1920329 ,
        -0.0106803 , -0.07432184,  0.003447  ,  0.11814891, -0.07613041,
         0.01972482, -0.08320747,  0.13414364,  0.00047702, -0.15275721,
         0.09923995, -0.06166402]], dtyp

# Layers are recursively composable

In [15]:
class MLPBlock(keras.layers.Layer):
  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(32)
    self.linear_2 = Linear(32)
    self.linear_3 = Linear(1)
  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

In [16]:
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3,64)))

In [17]:
mlp.weights

[<tf.Variable 'mlp_block/linear_5/Variable:0' shape=(64, 32) dtype=float32, numpy=
 array([[ 0.03181227, -0.03552845, -0.00532066, ...,  0.12079153,
         -0.07475913,  0.0099043 ],
        [ 0.0015341 , -0.02581516, -0.03239488, ...,  0.0370282 ,
          0.02811593, -0.03380568],
        [-0.00715649,  0.0582541 ,  0.04256558, ...,  0.0377348 ,
          0.10212301, -0.03072884],
        ...,
        [ 0.05020842, -0.10764688,  0.00189259, ..., -0.0117731 ,
          0.02547028, -0.06487002],
        [-0.00168144, -0.01505039, -0.0209573 , ..., -0.01630844,
         -0.0164077 , -0.07637613],
        [-0.05147887,  0.09903259,  0.01605012, ...,  0.02922877,
          0.0156203 ,  0.07475672]], dtype=float32)>,
 <tf.Variable 'mlp_block/linear_5/Variable:0' shape=(32,) dtype=float32, numpy=
 array([-0.01699487,  0.01961638,  0.04468011, -0.07827999,  0.06531694,
         0.04750273, -0.01846105, -0.04750106,  0.04930599,  0.03066118,
        -0.13191342, -0.00432053,  0.04816367,  

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎